In [ ]:
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs


import sys
sys.path.append('../')

from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
startyear = 1956
endyear   = 2000

Nax = 5

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0)#260-360)
axex = [245.8,251,-75.2,-74.4]
axex2 = [245.48,246.17,-75.11,-74.9]
#axinsex1 = [245.65,246.05,-75.1,-75]
#axinsex2 = [246.15,246.45,-74.73,-74.53]

In [ ]:
def makebackground(ax,axex):
    cmap = plt.get_cmap('ocean')
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(3400,3800),y=slice(7700,8150))
    mask = xr.where(ds.mask==1,2,ds.mask)
    #mask[:] = xr.where(mask==1,2,mask)
    ds = add_lonlat(ds)
    ax.pcolormesh(ds.lon,ds.lat,mask,cmap=cmap,vmin=-4,vmax=3,transform=ccrs.PlateCarree(),shading='auto')
    ax.set_extent(axex,crs=ccrs.PlateCarree())
    gl = ax.gridlines()
    gl.xlocator = mpl.ticker.FixedLocator(np.arange(-180,179,1))
    gl.ylocator = mpl.ticker.FixedLocator(np.arange(-89,89,.2))

In [ ]:
def plotmelt(nax,lon,lat,melt,title):
    ax = fig.add_subplot(2,Nax,nax,projection=proj)
    makebackground(ax,axex)
    ax.set_extent(axex,crs=ccrs.PlateCarree())
    im = ax.pcolormesh(lon,lat,np.where(melt==0,np.nan,melt),cmap='inferno',norm=mpl.colors.LogNorm(vmin=1,vmax=100),shading='nearest',transform=ccrs.PlateCarree())
    #cbar = plt.colorbar(im,ax=ax,orientation='horizontal')
    #cbar.set_ticks([1,10,100])
    #cbar.set_ticklabels([1,10,100])
    ax.set_title(title)
    axins = inset_axes(ax,width="0%",height="0%",loc='lower right',borderpad=0,axes_class=cartopy.mpl.geoaxes.GeoAxes,axes_kwargs=dict(map_projection=proj))    
    axins.set_extent(axex2,crs=ccrs.PlateCarree())
    ax.indicate_inset_zoom(axins, edgecolor="k",lw=2)
    
    ax = fig.add_subplot(2,Nax,nax+Nax,projection=proj)
    makebackground(ax,axex2)
    ax.set_extent(axex2,crs=ccrs.PlateCarree())
    im = ax.pcolormesh(lon,lat,np.where(melt==0,np.nan,melt),cmap='inferno',norm=mpl.colors.LogNorm(vmin=1,vmax=100),shading='nearest',transform=ccrs.PlateCarree())
    #cbar = plt.colorbar(im,ax=ax,orientation='horizontal')
    #cbar.set_ticks([1,10,100])
    #cbar.set_ticklabels([1,10,100])
    #ax.set_title(title)
    
    #Add arrows
    transform = ccrs.PlateCarree()._as_mpl_transform(ax)
    ax.annotate('',(245.7,-75.05),xytext=(245.55,-75.07),xycoords=transform,arrowprops=dict(facecolor='tab:blue',edgecolor='tab:blue',arrowstyle="simple"))
    ax.annotate('',(245.8,-75.08),xytext=(245.85,-75.11),xycoords=transform,arrowprops=dict(facecolor='tab:red',edgecolor='tab:red',arrowstyle="simple"))
    ax.annotate('',(246.02,-75.04),xytext=(246.13,-75.07),xycoords=transform,arrowprops=dict(facecolor='tab:blue',edgecolor='tab:blue',arrowstyle="simple"))

    
    print(np.nanmean(np.where(melt==0,np.nan,melt)),np.nanmax(np.where(melt==0,np.nan,melt)))
    return im
    
    """
    # inset axes....
    axins = inset_axes(ax,width="100%",height="100%",loc='lower right',borderpad=0,axes_class=cartopy.mpl.geoaxes.GeoAxes,axes_kwargs=dict(map_projection=proj))    
    axins.pcolormesh(lon,lat,np.where(melt==0,np.nan,melt),cmap='inferno',norm=mpl.colors.LogNorm(vmin=1,vmax=200),shading='nearest',transform=ccrs.PlateCarree())
    axins.set_extent(axinsex1,crs=ccrs.PlateCarree())
    axins.set_xticklabels('')
    axins.set_yticklabels('')
    ax.indicate_inset_zoom(axins, edgecolor="k",lw=2)
    
    # inset axes....
    axins = inset_axes(ax,width="20%",height="40%",loc='upper left',borderpad=0,axes_class=cartopy.mpl.geoaxes.GeoAxes,axes_kwargs=dict(map_projection=proj))    
    axins.pcolormesh(lon,lat,np.where(melt==0,np.nan,melt),cmap='inferno',norm=mpl.colors.LogNorm(vmin=1,vmax=200),shading='nearest',transform=ccrs.PlateCarree())
    axins.set_extent(axinsex2,crs=ccrs.PlateCarree())
    axins.set_xticklabels('')
    axins.set_yticklabels('')
    ax.indicate_inset_zoom(axins, edgecolor="w",lw=2)
    """

In [ ]:
mpl.rc('figure.subplot',left=.01,right=.9,top=.9,bottom=.05,wspace=.02,hspace=.05)

fig = plt.figure(figsize=(7,3))

#Obs
ds = xr.open_dataset('../../../data/gourmelen/CrossonDotson.nc')
ds = add_lonlat(ds)
melt = ds.Band1
im = plotmelt(1,ds.lon,ds.lat,melt,'Altimetry')

#MITgcm
timep= slice(f"{startyear}-1-1",f"{endyear}-12-31")
ds = xr.open_dataset('../../../data/paulholland/melt.nc')
ds = ds.sel(LONGITUDE=slice(360-114.7,360-109.3),LATITUDE=slice(-75.3,-74.1),TIME=timep)
ds = ds.mean(dim='TIME')
lon   = (ds.LONGITUDE-360.).values
lat   = (ds.LATITUDE-.05).values
melt = ds.melt.values
im = plotmelt(2,lon,lat,melt,'MITgcm')

#2km
ds = xr.open_dataset('../../results/CrossDots_2.5_mitgcm_1956_2000_30.000.nc')
im = plotmelt(3,ds.lon,ds.lat,ds.melt,'Layer 2.5 km')

#1km
ds = xr.open_dataset('../../results/CrossDots_1.0_mitgcm_1956_2000_20.000.nc')
im = plotmelt(4,ds.lon,ds.lat,ds.melt,'Layer 1.0 km')

#500m
ds = xr.open_dataset('../../results/CrossDots_0.5_mitgcm_1956_2000_10.000.nc')
im = plotmelt(5,ds.lon,ds.lat,ds.melt,'Layer 0.5 km')

ax = fig.add_subplot(1,Nax,Nax)
ax.set_visible(False)
axins = inset_axes(ax,width="8%",height="100%",loc='lower left',bbox_to_anchor=(1.04, 0, 1, 1),bbox_transform=ax.transAxes,borderpad=0)
cbar = plt.colorbar(im, cax=axins,extend='both')
cbar.set_ticks([1,10,100])
cbar.set_ticklabels([1,10,100])
cbar.set_label('Melt rate [m/yr]')

plt.savefig('../../figures/draftplot_basalmelt.png',dpi=450)